In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps


In [2]:
# Import csv and save as dataframe
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Narovchat,RU,53.88,43.69,62.60,70,4,1.77,clear sky,0.0,0
1,1,Bluff,NZ,-46.60,168.33,50.59,78,99,21.99,overcast clouds,0.0,0
2,2,Ribeira Grande,PT,38.52,-28.70,73.40,73,20,8.05,few clouds,0.0,0
3,3,Merauke,ID,-8.47,140.33,76.78,84,100,13.65,overcast clouds,0.0,0
4,4,Jamestown,US,42.10,-79.24,69.80,94,90,9.17,thunderstorm,0.0,0


In [3]:
# Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:

# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the customer for the maximum temperature preference.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
raining = str(input("Do you want it to be raining? (yes/no) "))


# Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").
snowing = str(input("Do you want it to be snowing? (yes/no) "))


# Filter the dataset to find the cities that fit the criteria.
# THE CODE BELOW ASSUMES THAT THE USER WILL ENTER yes/no IN CORRECTLY.  
# IT WILL NOT VERIFIY THE DATA HAS BEEN ENTERED CORRECTLY

if raining == "yes":
    if snowing == "yes":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] > 0)]
    elif snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] > 0)]
elif snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]

preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Ribeira Grande,PT,38.52,-28.70,73.40,73,20,8.05,few clouds,0.0,0
20,20,Hilo,US,19.73,-155.09,73.40,78,90,4.70,overcast clouds,0.0,0
24,24,Missoula,US,47.02,-113.80,73.00,56,1,4.74,clear sky,0.0,0
25,25,Lompoc,US,34.64,-120.46,71.60,60,0,5.82,clear sky,0.0,0
38,38,Vila Franca Do Campo,PT,37.72,-25.43,71.60,64,75,8.05,broken clouds,0.0,0
39,39,Baft,IR,29.23,56.61,74.64,21,26,13.69,scattered clouds,0.0,0
77,77,Avarua,CK,-21.21,-159.78,73.40,78,100,3.36,light rain,0.0,0
88,88,Shahrud,IR,36.42,54.98,72.10,53,28,15.52,light rain,0.0,0
89,89,Castro,BR,-24.79,-50.01,70.25,61,35,12.73,scattered clouds,0.0,0
148,148,Yellowknife,CA,62.46,-114.35,73.00,60,75,8.05,broken clouds,0.0,0


In [4]:
# Add the cities to a marker layer map with a pop-up marker for each city that includes:
# Hotel name
# City
# Country
# Current weather description with the maximum temperature

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""


# Set parameters to search for a hotel.
from config import g_key
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
 


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [5]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ribeira Grande,PT,73.4,few clouds,38.52,-28.70,Quinta da Meia Eira
20,Hilo,US,73.4,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
24,Missoula,US,73.0,clear sky,47.02,-113.80,Gold Creek Cabin
25,Lompoc,US,71.6,clear sky,34.64,-120.46,Holiday Inn Express Lompoc
38,Vila Franca Do Campo,PT,71.6,broken clouds,37.72,-25.43,Pestana Bahia Praia


In [6]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [7]:
# Add a marker layer map and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv.

output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")